Questão 1: Crie uma aplicação simples em FastAPI que utilize o modelo GPT-2 da HuggingFace para gerar textos a partir de uma entrada fornecida via requisição HTTP.

O aplicativo deve:

Receber uma frase de entrada como JSON.
Utilizar a biblioteca transformers do HuggingFace para gerar um texto de saída.
Retornar o texto gerado em uma resposta HTTP.
O que é esperado:

O aplicativo deve gerar uma continuação de texto a partir de uma frase de entrada e retornar a resposta formatada como JSON.


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline, set_seed

app = FastAPI()

# Carregar o pipeline GPT-2
text_generator = pipeline("text-generation", model="gpt2", device=0)
set_seed(42)  

class InputText(BaseModel):
    text: str

@app.post("/generate-text/")
def generate_text(input_text: InputText):
    if not input_text.text:
        raise HTTPException(status_code=400, detail="Input text is required.")
    # Gerar texto
    generated = text_generator(input_text.text, max_length=50, num_return_sequences=1)
    return {"input": input_text.text, "output": generated[0]["generated_text"]}


Questão 2: Crie um aplicativo FastAPI que utiliza o modelo de tradução Helsinki-NLP/opus-mt-en-fr da HuggingFace para traduzir textos do inglês para o francês.

A aplicação deve:

Receber um texto em inglês via uma requisição HTTP.
Traduzir o texto para o francês utilizando o modelo de tradução.
Retornar o texto traduzido em uma resposta JSON.
O que é esperado:

A API deve receber um texto em inglês e retornar sua tradução para o francês, processando tanto frases curtas quanto textos mais longos.


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI()

# Carregar o pipeline de tradução
translator = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr", device=0)

class TranslationInput(BaseModel):
    text: str

@app.post("/translate/")
def translate_text(input_text: TranslationInput):
    if not input_text.text:
        raise HTTPException(status_code=400, detail="Input text is required.")
    # Traduzir texto
    translated = translator(input_text.text, max_length=400)
    return {"input": input_text.text, "translation": translated[0]["translation_text"]}


Questão 3: Com base na API desenvolvida na Questão 2 (Parte1), explique as principais limitações do modelo de tradução utilizado.

Enumere e discuta:

Limitações quanto à precisão da tradução.
Desafios de tempo de resposta e desempenho em grande escala.
Restrições de custo e escalabilidade.
Limitações na tradução de gírias, expressões idiomáticas ou linguagem de contexto.

R.

Precisão da Tradução:
Não obtive a validação mais correta, visto que tive de traduzir do francês para o inglês novamente, mas os textos estavam precisos.

Desenpenho:
Aumento do tempo de resposta com textos longos se aplicaria caso quisessemos traduzir um livro, testei com vários paragrafos e obtive resposta instantânea.

Custo e Escalabilidade:
Hardware para operações em larga escala seria um "problema" caso estivessemos em alguma cloud, custos associados a servidores com GPUs.

Contexto:
Ao inserir gírias e abreviações como (Chill out, goat, dope, on fleek, you"u", because"bc", easy"ez") o modelo traduziu corretamente, diferente do que acontece em tradutores normais.


Questão 4: Com base no modelo GPT-2 utilizado na Questão 1 (Parte 1), explique as principais limitações do modelo no contexto da geração de texto.

Discuta:

A coerência do texto gerado.
Possíveis falhas ou incoerências geradas por LLMs.
Desempenho e questões de latência.
Limitações na geração de conteúdo apropriado.

R. 

Coerência do Texto:
O texto certamente vai divergir ou ser incoerente no que foi solicitado, mesmo com um prompt simples o modelo vai escrever com base no seu embeding que não é tão bem calibrado.

Falhas em Geração:
percebi alguamas falhas na pontuação como uma única contra barra sem muito sentido, ou uma única aspas duplas, tabém sem contexto.

Desempenho:
Viés garantido. final de geração de texto aleatório.
Boa rapidez na resposta.☻

Geração de Conteúdo Apropriado:

Falta de controle sobre o tom, por ser um modelo com um abismo de discrepância em relação aos atuais, é certeza de não se o conteúdo totalmente correto, 

Exemplo de resposta que o modelo me forneceu com um conteúdo não tão apropriado.
{
    "input": "Say the names of countrys in Brazil?",     
    "output": "Say the names of countrys in Brazil?\n\nThe United States is the only country where a man wants his wife or sister to have a baby and wants them to be free, a person in Brazil told me.\n\nIn the case"
}

Questão 1: Desenvolva um protótipo utilizando LangChain que simule um chatbot simples com Fake LLM.

A aplicação deve:

Receber um input de texto via FastAPI.
Retornar uma resposta simulada pelo Fake LLM.
O que é esperado:

O protótipo deve simular um chatbot básico que responde a perguntas pré-definidas. 
A arquitetura deve ser simples, e você deve explicar a importância de usar Fake LLM para testes rápidos. 
Desenhe um diagrama simples da arquitetura do aplicativo, detalhando as principais etapas do fluxo de dados.


In [ ]:
from fastapi import FastAPI, Request
from langchain.llms import FakeListLLM
from pydantic import BaseModel

app = FastAPI()

# Respostas pré-definidas
responses = [
    "Olá! Como posso ajudá-lo hoje?",
    "Estou bem, obrigado por perguntar!",
    "Tenha um ótimo dia!"
]

# Configurar o FakeLLM
fake_llm = FakeListLLM(responses=responses)

class InputText(BaseModel):
    text: str

@app.post("/chatbot/")
async def chatbot(input: InputText):
    response = fake_llm(input.text)
    return {"response": response}


+------------+          +----------+         +-----------+
| User Input | ----->   | FastAPI  | ----->  | Fake LLM  |
+------------+          +----------+         +-----------+
      ^                                                    |
      +----------------------------------------------------+


Questão 2: Desenvolva um aplicativo que utilize LangChain para integrar a API da OpenAI.

O aplicativo deve:

Receber um texto em inglês via FastAPI.
Traduzir o texto para o francês utilizando um modelo da OpenAI via LangChain.
Retornar o texto traduzido em uma resposta JSON.
O que é esperado:

O aplicativo deve funcionar como uma API de tradução, semelhante à questão 2 (Parte 1), mas utilizando a OpenAI via LangChain. 
A aplicação deve gerenciar as chamadas à API da OpenAI e retornar a tradução com baixa latência.
Forneça um diagrama da arquitetura da aplicação, destacando os componentes principais, como FastAPI, LangChain, e OpenAI.


In [ ]:
from fastapi import FastAPI, HTTPException
from langchain.schema import HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import os
from pydantic import BaseModel
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

app = FastAPI()

# Modelo de entrada
class TextInput(BaseModel):
    text: str

# Configuração do OpenAI LangChain
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.7
)

@app.post("/translate/")
async def translate_to_french(input: TextInput):
    """
    Traduz texto do inglês para o francês usando OpenAI via LangChain
    """
    try:
        # Criar o prompt e obter a tradução
        message = HumanMessage(content=f"Translate this to French: {input.text}")
        response = llm.invoke([message])
        return {"original_text": input.text, "translated_text": response.content}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



+------------+          +----------+          +-----------+         +----------+
| User Input | ----->   | FastAPI  | ----->   | LangChain | ----->  | OpenAI   |
+------------+          +----------+          +-----------+         +----------+
      ^                                                                            |
      +----------------------------------------------------------------------------+


Questão 3: Crie uma API semelhante à Questão 2 (Parte 2), mas que utilize o modelo Helsinki-NLP/opus-mt-en-de da HuggingFace para traduzir textos do inglês para o alemão.

A aplicação deve:

Receber um texto em inglês via FastAPI.
Utilizar o LangChain para gerenciar as chamadas ao modelo HuggingFace.
Retornar o texto traduzido para o alemão como resposta JSON.
O que é esperado:

O objetivo é que a aplicação funcione de maneira semelhante às Questões 2 (Parte 1) e 2 (Parte 2), mas desta vez integrando LangChain com HuggingFace. 
O modelo a ser utilizado deve ser o Helsinki-NLP/opus-mt-en-de.
Forneça um diagrama detalhado da arquitetura da aplicação, destacando as interações entre FastAPI, LangChain, e HuggingFace.


In [ ]:
from fastapi import FastAPI, HTTPException
from transformers import pipeline
from pydantic import BaseModel

app = FastAPI()

# Carregar o modelo de tradução
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de", device=0)

# Modelo de entrada
class TextInput(BaseModel):
    text: str

@app.post("/translate/")
async def translate_to_german(input: TextInput):
    """
    Traduz texto do inglês para o alemão usando Helsinki-NLP diretamente com transformers.
    """
    try:
        # Obter a tradução
        result = translator(input.text, max_length=400)
        translated_text = result[0]["translation_text"]
        return {"original_text": input.text, "translated_text": translated_text}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))




+------------+          +----------+          +-----------+         +---------------+
| User Input | ----->   | FastAPI  | ----->   | LangChain | ----->  | HuggingFace   |
+------------+          +----------+          +-----------+         +---------------+
      ^                                                                                 |
      +---------------------------------------------------------------------------------+


Questão 4: Com base na implementação da Questão 2 (Parte 2), explique as principais limitações de utilizar LangChain para integrar a API da OpenAI.

Discuta os seguintes aspectos:

Latência de resposta.
Limites de uso da API da OpenAI.
Desafios de escalabilidade e custo.
Qualidade das traduções geradas em comparação com outros modelos.



In [1]:
pip install pandas


  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

df = pd.read_csv('../data/quest4.csv', sep=';')
print(df)

                   Aspecto                                          Descrição
0     Latência de resposta  É importante lembrar que terá o processamento ...
1    Limites de uso da API  O limite de uso é conforme o modelo selecionad...
2   Escalabilidade e custo  A melhor estratégia é usar o modelo necessário...
3  Qualidade das traduções  os modelos da openai tem uma qualidade impecáv...


In [ ]:
Questões 4 e 5: Limitações
LangChain + OpenAI:

Latência de Resposta: As chamadas à API da OpenAI podem ser lentas dependendo da carga do servidor.
Limites de Uso: A OpenAI impõe limites de requisição e custos por token, dificultando a escalabilidade.
Custo: O uso frequente da API pode ser caro para aplicações em larga escala.
Qualidade: Traduções podem ser inconsistentes comparadas a modelos especializados.
LangChain + HuggingFace:

Desempenho: Processamento local pode ser mais lento sem hardware apropriado (GPU).
Recursos Computacionais: Modelos grandes exigem mais memória e poder computacional.
Flexibilidade: Ajustar o modelo requer conhecimento técnico avançado.
Comparação com API: A API HuggingFace pode ser mais simples, mas é paga para uso em larga escala.


Questão 5: Com base na aplicação desenvolvida na 3 (Parte 2), explique as limitações de usar LangChain para integrar o modelo HuggingFace de tradução. 

Discuta aspectos como:

Desempenho e tempo de resposta.
Consumo de recursos computacionais.
Possíveis limitações no ajuste fino do modelo.
Comparação com o uso direto da API HuggingFace.



Questão 6: Com base nas questões 1-2 (Parte 1) e 2-3 (Parte 2), desenvolva uma tabela comparativa que aborde os seguintes critérios:

Facilidade de uso/configuração.
Latência e desempenho.
Flexibilidade para diferentes modelos.
Custo e escalabilidade.
Adequação para protótipos versus aplicações em produção.
A comparação deve ser apresentada em formato de tabela, com colunas dedicadas a cada critério e linhas comparando FastAPI puro com LangChain.


Questão 6: Tabela Comparativa
Critério	FastAPI Puro	LangChain
Facilidade de Uso	Simples e direto	Mais configuração, mas modular.
Latência	Depende da integração	Pode aumentar devido à camada adicional.
Flexibilidade	Limitada a bibliotecas	Suporte para várias LLMs e integrações.
Custo	Baixo	Pode ser alto dependendo das integrações.
Adequação	Bom para protótipos	Excelente para produção e flexibilidade.